<a href="https://colab.research.google.com/github/angelinasinaga/Indonesian_Named-entity_Recognition/blob/Angelina/Pasupasu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. LOAD DATA** ##

**1.1 Import Libraries**

In [ ]:
#Data analysis   
import pandas as pd 
import numpy as np
import csv
import nltk
import tensorflow as tf

In [ ]:
#Data visualisation 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
from matplotlib import ticker
%config InlineBackend.figure_format = 'svg'

In [ ]:
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# from sklearn_crfsuite import CRF, scorers, metrics
# import sklearn_crfsuite
# from sklearn_crfsuite import scorers
# from sklearn_crfsuite import metrics
# from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import scipy.stats
import eli5

**1.2 Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
list_kalimat = []
nomor_kalimat = 1

# open file in read mode
with open('/content/drive/MyDrive/Semester 7/Proyek PBA/Dataset/SINGGALANG.tsv', encoding='UTF-8') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = csv.reader(read_obj, delimiter='\t', quotechar=None) #https://www.petanikode.com/python-csv/ 
# https://docs.python.org/id/3.9/library/csv.html
# Menginstruksikan reader untuk tidak melakukan pemrosesan khusus terhadap karakter kutipan
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        if len(row) == 0:
            nomor_kalimat += 1 
        else:
            tmp_row= row
            tmp_row.append(nomor_kalimat)
            list_kalimat.append(tmp_row)

dataset = pd.DataFrame(list_kalimat, columns=['token', 'entitas bernama', 'kalimat'])

In [ ]:
dataset.head()

,token,entitas bernama,kalimat
0,Ia,O,1.0
1,menjabat,O,1.0
2,sebagai,O,1.0
3,Presiden,O,1.0
4,ketiga,O,1.0


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116307 entries, 0 to 116306
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   token            116307 non-null  object 
 1   entitas bernama  116307 non-null  object 
 2   kalimat          116306 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.7+ MB


## **2. Exploring / Visualisasi Data (Analisis Data)** ##

Membangun kelas sederhana untuk menggabungkan kata-kata ke dalam kalimat. 

In [ ]:
print("Jumlah kalimat: ", len(dataset.groupby(['kalimat'])))
words = list(set(dataset["token"].values))
n_words = len(words)
print("Jumlah kata unik : ", n_words)
tags = list(set(dataset["entitas bernama"].values))
print("Entitas bernama:", tags)
n_tags = len(tags)
print("Jumlah entitas bernama: ", n_tags)

Jumlah kalimat:  3818
Jumlah kata unik :  17391
Entitas bernama: [3818, 'O', 'Organisation', 'Place', 'Person']
Jumlah entitas bernama:  5


In [ ]:
#Words tagged as Place
dataset.loc[dataset['entitas bernama'] == 'Place', 'token'].head()

5        Mesir
29       Mesir
47       Mesir
68    Kerajaan
70       Kairo
Name: token, dtype: object

In [ ]:
#Words tagged as Person
dataset.loc[dataset['entitas bernama'] == 'Person', 'token'].head()

38     Sadat
100    Anwar
101    Sadat
105    Hafez
107    Assad
Name: token, dtype: object

In [ ]:
#Words tagged as Organisation
dataset.loc[dataset['entitas bernama'] == 'Organisation', 'token'].head()

568    Universitas
569      Indonesia
711     Independen
778    Universitas
779         Gadjah
Name: token, dtype: object

In [ ]:
#Words tagged as O
dataset.loc[dataset['entitas bernama'] == 'O', 'token'].head()

0          Ia
1    menjabat
2     sebagai
3    Presiden
4      ketiga
Name: token, dtype: object

**Menghitung Jumlah Entitas Bernama**

In [ ]:
dataframe = dataset.groupby("entitas bernama" )
dataframe["entitas bernama"].count() 

entitas bernama
3818                 1
O               103251
Organisation       804
Person            2358
Place             9893
Name: entitas bernama, dtype: int64